In [ ]:
! pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 re

In [ ]:
import pandas as pd
df = pd.read_csv('/content/all-data.csv', encoding='latin-1')
df.columns = ['Sentiment', 'Headline']

# Quick look
print(df.head())
print(df['Sentiment'].value_counts())

  Sentiment                                           Headline
0   neutral  Technopolis plans to develop in stages an area...
1  negative  The international electronic industry company ...
2  positive  With the new production plant the company woul...
3  positive  According to the company 's updated strategy f...
4  positive  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
Sentiment
neutral     2878
positive    1363
negative     604
Name: count, dtype: int64


In [ ]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)
print("After removing duplicates:", df.shape)

After removing duplicates: (4839, 2)


In [ ]:
import pandas as pd
import re
import spacy

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

def preprocess_text_spacy(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)

    # 3. Remove emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002700-\U000027BF"  # dingbats
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # 4. Process text with spaCy
    doc = nlp(text)

    # 5. Filter tokens: remove stop words, punctuations, numbers, etc.
    tokens = [token.lemma_ for token in doc
              if not token.is_stop
              and not token.is_punct
              and not token.like_num
              and token.is_alpha]

    return tokens

# Apply preprocessing
df['cleaned_tokens'] = df['Headline'].apply(preprocess_text_spacy)

# Optional: join back into text string
df['cleaned_text'] = df['cleaned_tokens'].apply(lambda tokens: ' '.join(tokens))

# Display result
print(df[['Headline', 'cleaned_tokens', 'cleaned_text']].head())


                                            Headline  \
0  Technopolis plans to develop in stages an area...   
1  The international electronic industry company ...   
2  With the new production plant the company woul...   
3  According to the company 's updated strategy f...   
4  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...   

                                      cleaned_tokens  \
0  [technopolis, plan, develop, stage, area, squa...   
1  [international, electronic, industry, company,...   
2  [new, production, plant, company, increase, ca...   
3  [accord, company, update, strategy, year, basw...   
4  [financing, aspocomp, growth, aspocomp, aggres...   

                                        cleaned_text  
0  technopolis plan develop stage area square met...  
1  international electronic industry company elco...  
2  new production plant company increase capacity...  
3  accord company update strategy year basware ta...  
4  financing aspocomp growth aspocomp aggressivel..

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Sentiment'])  # Maps: Negative=0, Neutral=1, Positive=2
print(dict(zip(le.classes_, le.transform(le.classes_))))

{'negative': 0, 'neutral': 1, 'positive': 2}


In [ ]:
# Download necessary NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Tag each document with a unique ID
tagged_data = [TaggedDocument(words=tokens, tags=[str(i)]) for i, tokens in enumerate(df['cleaned_tokens'])]

# Train a Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=40)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Infer vectors for each document
X_vectors = np.array([doc2vec_model.infer_vector(tokens) for tokens in df['cleaned_tokens']])
y = df['Label']


# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

# Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Accuracy
train_acc = accuracy_score(y_train, clf.predict(X_train))
test_acc = accuracy_score(y_test, clf.predict(X_test))

print(f"\n✅ Train Accuracy: {train_acc:.4f}")
print(f"✅ Test Accuracy: {test_acc:.4f}")


✅ Train Accuracy: 0.6451
✅ Test Accuracy: 0.6488


In [ ]:
df.head()

,Sentiment,Headline,cleaned_tokens,cleaned_text,Label
0,neutral,Technopolis plans to develop in stages an area...,"[technopolis, plan, develop, stage, area, squa...",technopolis plan develop stage area square met...,1
1,negative,The international electronic industry company ...,"[international, electronic, industry, company,...",international electronic industry company elco...,0
2,positive,With the new production plant the company woul...,"[new, production, plant, company, increase, ca...",new production plant company increase capacity...,2
3,positive,According to the company 's updated strategy f...,"[accord, company, update, strategy, year, basw...",accord company update strategy year basware ta...,2
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,"[financing, aspocomp, growth, aspocomp, aggres...",financing aspocomp growth aspocomp aggressivel...,2


In [ ]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import AdamW  # ✅ Use AdamW from PyTorch

# ✅ GPU Check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['cleaned_text'].tolist(), df['Label'].tolist(), test_size=0.2, random_state=42, stratify=df['Label']
)

# ✅ Compute class weights
unique_classes = np.unique(train_labels)
class_weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# ✅ Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# ✅ Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

# ✅ Create PyTorch Dataset
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

# ✅ Load BERT model with custom dropout to prevent overfitting
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=3, hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.3)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model.to(device)

# ✅ DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# ✅ Optimizer with lower learning rate and weight decay
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

# ✅ Loss function with class weights
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# ✅ Training loop
model.train()
epochs = 3
train_preds = []
train_targets = []

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        logits = outputs.logits
        loss = loss_fn(logits, batch['labels'])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        preds = torch.argmax(logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_targets.extend(batch['labels'].cpu().numpy())

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

# ✅ Evaluation on test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# ✅ Results
train_acc = accuracy_score(train_targets, train_preds)
test_acc = accuracy_score(all_labels, all_preds)

print("\n🧪 Train Accuracy:", train_acc)
print("\n✅ Test Accuracy:", test_acc)
print("\n📊 Classification Report:\n", classification_report(all_labels, all_preds, target_names=["Negative", "Neutral", "Positive"]))


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...


Epoch 2/3


Epoch 2: 100%|██████████| 242/242 [00:44<00:00,  5.42it/s, loss=0.318]



Epoch 3/3


Epoch 3: 100%|██████████| 242/242 [00:46<00:00,  5.23it/s, loss=1.28]



🧪 Train Accuracy: 0.6471196073366056

✅ Test Accuracy: 0.7603305785123967

📊 Classification Report:
               precision    recall  f1-score   support

    Negative       0.69      0.86      0.76       121
     Neutral       0.91      0.71      0.80       574
    Positive       0.61      0.82      0.70       273

    accuracy                           0.76       968
   macro avg       0.74      0.80      0.75       968
weighted avg       0.80      0.76      0.77       968



In [ ]:
from sklearn.metrics import accuracy_score

# ✅ Train Accuracy
model.eval()
train_preds = []
train_true = []

with torch.no_grad():
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        train_preds.extend(preds.cpu().numpy())
        train_true.extend(batch['labels'].cpu().numpy())

train_accuracy = accuracy_score(train_true, train_preds)
print("\n🧪 Train Accuracy:", train_accuracy)

# ✅ Test Accuracy (already included in your code)
test_accuracy = accuracy_score(all_labels, all_preds)
print("\n✅ Test Accuracy:", test_accuracy)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



🧪 Train Accuracy: 0.7951433738052183

✅ Test Accuracy: 0.7603305785123967


In [ ]:
def predict_sentiment(texts, model, tokenizer, device):

    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

    # Tokenize input texts
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
    encodings = {key: val.to(device) for key, val in encodings.items()}

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

    return [label_map[p.item()] for p in preds]


In [ ]:
# Example new sentences
new_sentences =[
    "The central bank holds interest rates steady.",
    "Company Z announces a leadership change with a new CEO.",
    "Market awaits key inflation data release this week.",
    "Q4 earnings season begins, with varied results expected.",
    "The government's new budget focuses on infrastructure spending.",
    "Oil prices remain stable amid balanced supply and demand.",
    "Technology stocks trade flat ahead of a major conference.",
    "Analysts publish their quarterly forecast for the S&P 500.",
    "The company plans to open a new factory in the coming year.",
    "A new trade agreement between two countries is under discussion.",
    "Company X completes the acquisition of a smaller firm.",
    "Market volatility subsides, but trading remains cautious.",
    "New regulations are introduced for the banking sector.",
    "The stock market closes with minor gains and losses.",
    "A large-scale infrastructure project is announced.",
    "The company's annual report shows a slight increase in revenue.",
    "Foreign exchange rates experience minimal fluctuation."
]
# Get predictions
predictions = predict_sentiment(new_sentences, model, tokenizer, device)

# Display
for text, pred in zip(new_sentences, predictions):
    print(f"📝 Sentence: {text}\n🔎 Predicted Sentiment: {pred}\n")


In [ ]:
# Define test sentences with their expected labels
test_data = [
    # Positive
    ("Company profits soar in the latest quarterly report.", "Positive"),
    ("The merger resulted in significant growth for both firms.", "Positive"),
    ("Investors are optimistic about the upcoming product launch.", "Positive"),
    ("The new policy boosts market confidence.", "Positive"),
    ("The company reports record-breaking annual revenue.", "Positive"),
    ("Tech stocks rally after positive earnings reports.", "Positive"),
    ("New partnership opens international markets for the firm.", "Positive"),
    ("Economic indicators show strong recovery signs.", "Positive"),
    ("The stock hits a 5-year high after strong performance.", "Positive"),
    ("Unemployment rate falls to a record low.", "Positive"),

    # Negative
    ("The company posts a major loss for the quarter.", "Negative"),
    ("Shares plunge after weak earnings report.", "Negative"),
    ("Layoffs announced as part of cost-cutting measures.", "Negative"),
    ("Regulatory issues hit the banking sector hard.", "Negative"),
    ("The CEO resigns amid controversy.", "Negative"),
    ("Weak consumer demand slows down production.", "Negative"),
    ("The new law could hurt small businesses.", "Negative"),
    ("Economic slowdown triggers fears of recession.", "Negative"),
    ("A major data breach exposes customer information.", "Negative"),
    ("Oil prices tumble due to oversupply concerns.", "Negative"),

    # Neutral
    ("The central bank holds interest rates steady.", "Neutral"),
    ("Company X announces its annual general meeting.", "Neutral"),
    ("The firm releases its regular quarterly update.", "Neutral"),
    ("Market opens flat with minimal movement.", "Neutral"),
    ("Traders await economic data before making moves.", "Neutral"),
    ("The currency exchange remains unchanged from yesterday.", "Neutral"),
    ("A board meeting is scheduled for next week.", "Neutral"),
    ("Company Y confirms completion of office relocation.", "Neutral"),
    ("A report on industrial output is due tomorrow.", "Neutral"),
    ("The industry conference is set to begin Monday.", "Neutral"),
]

# Separate the sentences and expected labels
sentences = [item[0] for item in test_data]
expected_labels = [item[1] for item in test_data]

# Get predictions from model
predicted_labels = predict_sentiment(sentences, model, tokenizer, device)

# Print comparison
for sentence, expected, predicted in zip(sentences, expected_labels, predicted_labels):
    print(f"📝 Sentence: {sentence}")
    print(f"✅ Expected: {expected}")
    print(f"🔎 Predicted: {predicted}")
    if expected.lower() != predicted.lower():
        print("❌ Incorrect Prediction\n")
    else:
        print("✅ Correct Prediction\n")


In [ ]:
# Test sentence
sentence = "Gold prices fall sharply as dollar strengthens amid recession fears."
expected_label = "Negative"  # Based on the content, inflation hurting consumers is negative

# Run prediction
prediction = predict_sentiment([sentence], model, tokenizer, device)[0]

# Display result
print(f"📝 Sentence: {sentence}")
print(f"✅ Expected: {expected_label}")
print(f"🔎 Predicted: {prediction}")
if prediction.lower() == expected_label.lower():
    print("✅ Correct Prediction\n")
else:
    print("❌ Incorrect Prediction\n")


In [ ]:
neutral_statements = [
    "The central bank maintains the current interest rate policy.",
    "Company X announces its quarterly earnings release date.",
    "A board meeting is scheduled for next Tuesday.",
    "The firm publishes its annual sustainability report.",
    "Market opens flat amid global economic uncertainty.",
    "The currency exchange rate remains stable today.",
    "Company Y completes relocation to its new headquarters.",
    "Investors await the upcoming monetary policy announcement.",
    "The CEO will deliver a keynote speech at the conference.",
    "Trading volume remains consistent with last week’s average.",
    "New regulations for data privacy go into effect next month.",
    "The stock exchange will be closed for a public holiday.",
    "A new report outlines trends in the technology sector.",
    "Company Z confirms the timeline for product launch.",
    "Economic data for the last quarter will be released Friday.",
    "The index shows minor fluctuations during the day’s session.",
    "Analysts maintain a hold rating on the stock.",
    "The real estate market remains largely unchanged.",
    "Company Q files its routine financial disclosures.",
    "The government presents its mid-year economic review."
]


In [ ]:
# Get predictions
predictions = predict_sentiment(neutral_statements, model, tokenizer, device)

# Display results
for sentence, prediction in zip(neutral_statements, predictions):
    expected = "Neutral"
    print(f"📝 Sentence: {sentence}")
    print(f"✅ Expected: {expected}")
    print(f"🔎 Predicted: {prediction}")
    if expected.lower() != prediction.lower():
        print("❌ Incorrect Prediction\n")
    else:
        print("✅ Correct Prediction\n")


In [ ]:
# 50 Neutral test statements
neutral_statements = [
    "The central bank keeps interest rates unchanged this quarter.",
    "Company A held its annual general meeting on Monday.",
    "The firm announced a restructuring of its board of directors.",
    "Market opened flat amid mixed global signals.",
    "Quarterly results are expected to be announced next week.",
    "The stock market remained closed on account of a public holiday.",
    "Company B updates its privacy policy for compliance.",
    "The central government is expected to unveil a new report.",
    "Exchange rates remain stable compared to the previous session.",
    "The firm confirms the timeline for the infrastructure project.",
    "A report on agricultural production is scheduled for release.",
    "Company X finalizes its employee handbook revision.",
    "Stock trading remains within a narrow range.",
    "Energy consumption figures for last month are now available.",
    "No significant change observed in commodity prices.",
    "A new tax regulation will come into effect next quarter.",
    "The ministry has released a public statement on the new plan.",
    "The IT department scheduled maintenance over the weekend.",
    "A quarterly update was posted on the company’s website.",
    "The company has signed a memorandum of understanding.",
    "A seminar on market trends was conducted by analysts.",
    "The government submitted its mid-year economic review.",
    "Company D held a press briefing regarding recent updates.",
    "The upcoming conference will feature international speakers.",
    "Trading activity slowed due to low market participation.",
    "The annual sustainability report has been published online.",
    "The finance ministry reviewed budget allocations this week.",
    "Company E has opened a new regional office.",
    "A status update was issued on the logistics project.",
    "No new policy decisions were taken during the meeting.",
    "Analysts maintain their neutral outlook on the sector.",
    "The new CEO attended his first official board meeting.",
    "The development project continues as per the original plan.",
    "The manufacturing unit resumed operations post inspection.",
    "Officials conducted a routine audit of corporate records.",
    "The shareholding pattern was disclosed to the exchange.",
    "Company F posted its recruitment drive schedule online.",
    "The taxation department updated documentation requirements.",
    "Public transport services operated normally despite rainfall.",
    "Investor presentations were held virtually on Friday.",
    "Monthly updates were shared with all stakeholders.",
    "The press release outlined key management appointments.",
    "Revenue figures matched analyst estimates this quarter.",
    "The company signed a non-binding letter of intent.",
    "Shipping schedules remain unaffected by recent events.",
    "Company G participated in the annual industry meet.",
    "A formal response is awaited on the pending proposal.",
    "The firm is reviewing its internal audit policy.",
    "No major movement seen in real estate prices.",
    "The export council presented trade figures to the media."
]

# Predict using your existing function
predictions = predict_sentiment(neutral_statements, model, tokenizer, device)

# Evaluate each prediction
for sentence, prediction in zip(neutral_statements, predictions):
    expected = "Neutral"
    print(f"📝 Sentence: {sentence}")
    print(f"✅ Expected: {expected}")
    print(f"🔎 Predicted: {prediction}")
    if expected.lower() != prediction.lower():
        print("❌ Incorrect Prediction\n")
    else:
        print("✅ Correct Prediction\n")


In [ ]:
# 50 Positive Sentences
positive_sentences = [
    "The company posted record-breaking profits this quarter.",
    "Investor confidence surged after the announcement.",
    "Shares rallied after the successful product launch.",
    "The new trade policy received positive feedback.",
    "Market sentiment improved significantly this week.",
    "Revenue doubled compared to the previous year.",
    "The firm exceeded analysts’ growth expectations.",
    "The economy shows strong signs of recovery.",
    "Unemployment rates fell to a decade low.",
    "The merger created significant shareholder value.",
    "Exports rose sharply boosting the trade surplus.",
    "Customer satisfaction scores reached an all-time high.",
    "Analysts upgraded the stock to a buy rating.",
    "Consumer demand increased steadily in the last month.",
    "The company secured a major international contract.",
    "The budget emphasized economic development and reforms.",
    "Sales figures surpassed projections by a large margin.",
    "The firm launched a successful innovation program.",
    "The currency strengthened after a policy change.",
    "The technology sector showed remarkable resilience.",
    "Interest in renewable energy investment is rising.",
    "The stock hit a 3-year high after earnings call.",
    "Management received praise for strategic decisions.",
    "Retail growth outpaced all forecasts this quarter.",
    "Strong earnings boosted investor morale.",
    "New markets opened for the brand overseas.",
    "The board announced a dividend increase.",
    "The infrastructure plan is expected to create jobs.",
    "The company received an industry excellence award.",
    "Positive consumer sentiment supported market recovery.",
    "The government's investment strategy was well-received.",
    "Quarterly gains pleased long-term investors.",
    "Brand loyalty contributed to rising revenues.",
    "The company reduced its debt significantly.",
    "Analysts report bullish trends in the stock.",
    "Tourism increased following improved travel rules.",
    "Foreign investment flowed into the energy sector.",
    "Productivity increased due to new automation systems.",
    "Hiring picked up across all departments.",
    "The product received excellent customer reviews.",
    "The economy grew faster than anticipated.",
    "The digital platform gained new subscribers.",
    "The new partnership is expected to boost growth.",
    "Shareholders welcomed the leadership transition.",
    "The acquisition deal closed successfully.",
    "The new policy favored small business growth.",
    "Employee engagement levels were at a peak.",
    "Exports hit a new high this quarter.",
    "Profits grew by 15% year-over-year.",
    "The fintech sector experienced a healthy rally.",
    "Consumer confidence reached its highest level in years."
]

# 50 Negative Sentences
negative_sentences = [
    "The company reported a significant loss this quarter.",
    "Investors reacted negatively to the earnings report.",
    "Layoffs were announced following declining sales.",
    "The stock plummeted after the CEO resigned.",
    "Economic indicators pointed to a slowdown.",
    "Revenue fell short of expectations.",
    "The merger failed to deliver expected synergies.",
    "Consumer demand weakened across all segments.",
    "The new law may hurt small businesses.",
    "The index closed in the red for a third day.",
    "Credit ratings were downgraded by major agencies.",
    "Production was halted due to a labor strike.",
    "Inflation hit a record high last month.",
    "Unemployment rates increased sharply.",
    "The company missed its revenue targets.",
    "Sales declined in key international markets.",
    "The budget deficit widened this year.",
    "Retailers struggled amid weak holiday sales.",
    "The firm's debt burden grew significantly.",
    "Supply chain issues disrupted delivery schedules.",
    "The government faced criticism for policy delays.",
    "The IT system suffered a major outage.",
    "Investor confidence dropped after the policy update.",
    "Profits fell amid rising operational costs.",
    "Trading was volatile after poor financial results.",
    "The acquisition led to integration challenges.",
    "Shares dipped due to negative market sentiment.",
    "Firms delayed investment due to economic uncertainty.",
    "Manufacturing output dropped for the second month.",
    "The lawsuit affected the brand’s reputation.",
    "Forecasts were revised downward for the next quarter.",
    "Prices surged, reducing consumer purchasing power.",
    "Exports dropped amid rising tariffs.",
    "Analysts issued a downgrade on the stock.",
    "Company faced backlash over environmental issues.",
    "Rising fuel prices strained transportation budgets.",
    "The firm failed to meet compliance requirements.",
    "The CEO admitted to financial mismanagement.",
    "Weak currency performance hurt importers.",
    "Falling consumer confidence affected spending.",
    "Shareholder concerns increased over governance issues.",
    "Tax increases dampened economic activity.",
    "The firm announced cost-cutting measures.",
    "Profit margins narrowed due to high input costs.",
    "Public protests disrupted business operations.",
    "The central bank expressed recession concerns.",
    "Construction delays affected project deadlines.",
    "Customer complaints rose sharply this year.",
    "The travel ban hurt tourism-related sectors.",
    "The board was criticized for poor oversight."
]

# Combine all for testing
all_test_sentences = positive_sentences + negative_sentences
expected_labels = ["Positive"] * 50 + ["Negative"] * 50

# Predict
predicted_labels = predict_sentiment(all_test_sentences, model, tokenizer, device)

# Track accuracy
incorrect_count = 0

# Show results
for sentence, expected, predicted in zip(all_test_sentences, expected_labels, predicted_labels):
    print(f"📝 Sentence: {sentence}")
    print(f"✅ Expected: {expected}")
    print(f"🔎 Predicted: {predicted}")
    if expected.lower() != predicted.lower():
        print("❌ Incorrect Prediction\n")
        incorrect_count += 1
    else:
        print("✅ Correct Prediction\n")

# Final tally
total = len(all_test_sentences)
correct = total - incorrect_count

print("📊 Evaluation Summary:")
print(f"✅ Total Sentences Tested: {total}")
print(f"✔️ Correct Predictions: {correct}")
print(f"❌ Incorrect Predictions: {incorrect_count}")


In [ ]:
# Longer statements (3–4 sentences each) with expected sentiment
long_statements = [
    # Positive
    ("The company reported strong financial results this quarter, exceeding analysts' expectations. "
     "Revenue grew by 20%, driven by increased demand in both domestic and international markets. "
     "Leadership expressed optimism about future performance and announced plans for expansion. "
     "Investors reacted positively, leading to a significant stock price increase.",
     "Positive"),

    ("After launching its new line of products, customer reviews have been overwhelmingly positive. "
     "Sales have surged across all regions, and the brand has gained substantial social media traction. "
     "Management confirmed increased hiring plans to support growth. "
     "Analysts upgraded their forecasts for the upcoming fiscal year.",
     "Positive"),

    # Negative
    ("The company faced backlash after reporting a data breach that exposed customer information. "
     "This led to a drop in customer trust and declining user engagement across its platform. "
     "To make matters worse, legal challenges were initiated by multiple consumer rights groups. "
     "Share prices fell sharply in response.",
     "Negative"),

    ("Quarterly reports showed a major decline in profits, primarily due to rising production costs. "
     "The CEO acknowledged operational inefficiencies and announced immediate cost-cutting measures. "
     "Several departments are expected to face layoffs. "
     "Investor sentiment turned sour, causing the stock to tumble.",
     "Negative"),

    # Neutral
    ("The company held its annual general meeting last Friday, where the board reviewed previous goals and set new ones. "
     "Shareholders were briefed on ongoing projects and long-term plans. "
     "No major announcements were made, and the stock remained stable throughout the day. "
     "The next report is expected in the following quarter.",
     "Neutral"),

    ("Economic data released this morning showed consistent numbers across key sectors. "
     "The inflation rate remained unchanged, and the central bank chose to maintain its interest rates. "
     "Markets reacted mildly, with minimal movement observed during early trading hours. "
     "Experts believe further updates will be available after the global summit next week.",
     "Neutral"),

    # More examples
    ("Despite economic challenges, the company managed to maintain a steady growth trajectory. "
     "Cost optimization strategies helped improve margins slightly. "
     "While revenue did not significantly increase, losses were contained. "
     "Management remains cautiously optimistic about the future.",
     "Neutral"),

    ("Following the approval of the new infrastructure bill, several construction companies saw a sharp rise in project allocations. "
     "Local governments praised the long-term benefits of the bill. "
     "Experts suggest it may help boost employment and stimulate regional economies. "
     "The announcement was well-received by market participants.",
     "Positive"),

    ("The company’s software upgrade was released with several bugs, causing service disruptions for many users. "
     "Multiple complaints were lodged, and support teams struggled to respond in time. "
     "The CTO apologized publicly and promised a fix within 48 hours. "
     "However, the negative feedback affected the company’s online ratings.",
     "Negative"),

    ("A new international summit was announced to address trade imbalance and promote collaboration. "
     "Delegates from over 40 countries are expected to attend the event. "
     "No immediate policy outcomes are expected, but dialogues will be initiated. "
     "The summit is scheduled for next month in Geneva.",
     "Neutral")
]

# Extract sentences and labels
long_texts = [item[0] for item in long_statements]
expected_labels = [item[1] for item in long_statements]

# Predict using your model
predicted_labels = predict_sentiment(long_texts, model, tokenizer, device)

# Evaluation
incorrect_count = 0

for sentence, expected, predicted in zip(long_texts, expected_labels, predicted_labels):
    print(f"📝 Text: {sentence}")
    print(f"✅ Expected: {expected}")
    print(f"🔎 Predicted: {predicted}")
    if expected.lower() != predicted.lower():
        print("❌ Incorrect Prediction\n")
        incorrect_count += 1
    else:
        print("✅ Correct Prediction\n")

# Final summary
total = len(long_texts)
print("📊 Long Statement Evaluation Summary:")
print(f"✅ Total Tested: {total}")
print(f"✔️ Correct: {total - incorrect_count}")
print(f"❌ Incorrect: {incorrect_count}")


📝 Text: The company reported strong financial results this quarter, exceeding analysts' expectations. Revenue grew by 20%, driven by increased demand in both domestic and international markets. Leadership expressed optimism about future performance and announced plans for expansion. Investors reacted positively, leading to a significant stock price increase.
✅ Expected: Positive
🔎 Predicted: Positive
✅ Correct Prediction

📝 Text: After launching its new line of products, customer reviews have been overwhelmingly positive. Sales have surged across all regions, and the brand has gained substantial social media traction. Management confirmed increased hiring plans to support growth. Analysts upgraded their forecasts for the upcoming fiscal year.
✅ Expected: Positive
🔎 Predicted: Positive
✅ Correct Prediction

📝 Text: The company faced backlash after reporting a data breach that exposed customer information. This led to a drop in customer trust and declining user engagement across its plat

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
paragraph_statements = [
    # Positive
    ("The company reported outstanding performance in the last quarter. Revenue grew by 30%, largely due to strong customer retention and increased demand for their flagship product. "
     "The leadership team announced bonuses and expanded hiring to meet growing needs. Analysts expressed confidence in the company's upward trajectory.",
     "Positive"),

    ("Following the successful launch of its new service, the company saw record-breaking user registrations within a week. "
     "Positive reviews flooded in, praising its user interface and performance. "
     "Investors responded with enthusiasm, pushing share prices higher. Executives confirmed further innovation in the pipeline.",
     "Positive"),

    ("The tech startup secured a major funding round from global investors. This will enable the company to scale its operations across multiple continents. "
     "The leadership emphasized sustainable growth and product development. The market responded favorably to the announcement.",
     "Positive"),

    ("Sales of electric vehicles reached a new peak this quarter. Consumer interest in eco-friendly transportation continues to rise. "
     "The government’s incentives have further boosted adoption. Industry leaders believe this trend will sustain long-term momentum.",
     "Positive"),

    ("The company has launched a new mentorship program for employees. Early feedback shows increased job satisfaction and productivity. "
     "The initiative is expected to improve employee retention rates. It reflects a strong internal culture and commitment to growth.",
     "Positive"),

    # Negative
    ("The company reported a disappointing earnings result, falling short of revenue estimates. Sales declined due to weak demand and supply chain disruptions. "
     "Investors expressed concern about the lack of strategic direction. The stock fell sharply in after-hours trading.",
     "Negative"),

    ("Consumer complaints have risen significantly over the past three months. Delays in delivery and customer service issues were frequently cited. "
     "Brand trust has taken a hit, according to recent surveys. Management has yet to provide a concrete response.",
     "Negative"),

    ("A major cybersecurity breach affected thousands of customer accounts. Sensitive information may have been compromised. "
     "The company is under investigation and faces potential legal consequences. Public backlash is growing by the hour.",
     "Negative"),

    ("The manufacturing plant was temporarily shut down after a worker strike. Production has halted, and no agreement has been reached yet. "
     "This disruption could affect distribution schedules nationwide. Analysts predict a decline in quarterly output.",
     "Negative"),

    ("The retail giant closed several underperforming outlets due to financial losses. This decision affects hundreds of employees. "
     "Stakeholders have raised concerns over the company’s long-term strategy. The company’s stock dipped following the announcement.",
     "Negative"),

    # Neutral
    ("The central bank held a press conference today to discuss macroeconomic trends. Officials confirmed that interest rates will remain unchanged. "
     "They emphasized the importance of monitoring inflation and employment data. Markets showed little reaction to the announcement.",
     "Neutral"),

    ("The company released its mid-year performance report. While figures remained consistent with earlier forecasts, no significant growth or decline was observed. "
     "Leadership confirmed that the organization is on track to meet its annual targets. No changes were announced to existing policies.",
     "Neutral"),

    ("A government panel has been formed to evaluate transportation reforms. The panel will meet weekly over the next two months. "
     "Recommendations will be made to improve efficiency and reduce congestion. No official decisions have been made yet.",
     "Neutral"),

    ("The annual general meeting took place on Wednesday with all board members present. The company presented its financial summary and answered stakeholder questions. "
     "There were no major announcements or shifts in company strategy. Media coverage of the event remained minimal.",
     "Neutral"),

    ("An industry conference is being held in Singapore this week. Delegates from around the world have gathered to discuss future trends. "
     "Workshops and networking sessions are scheduled throughout the event. Outcomes from the event are expected to be published next month.",
     "Neutral"),

    ("The government released a statement regarding recent rainfall patterns. Officials said current data does not warrant emergency action. "
     "Weather monitoring will continue as usual. Citizens were advised to stay informed through local channels.",
     "Neutral"),

    ("The software update was released as planned. Users noted minor changes in performance, with no critical issues reported. "
     "Technical teams are monitoring feedback for further tweaks. The next major update is scheduled for later this year.",
     "Neutral"),

    ("The firm has entered into a non-binding agreement with a logistics provider. The agreement outlines basic terms of cooperation. "
     "Further discussions are planned before final decisions are made. Neither party has made formal commitments yet.",
     "Neutral"),

    ("Company representatives attended a local job fair last weekend. Various departments showcased open roles and training programs. "
     "The response from applicants was moderate. Final recruitment numbers will be released soon.",
     "Neutral"),

    ("A new report was published analyzing population trends in urban areas. The study included data from 10 major cities. "
     "While growth patterns varied by region, overall trends remained within expected ranges. Policymakers are reviewing the findings.",
     "Neutral")
]

# Split inputs and expected labels
long_paragraphs = [item[0] for item in paragraph_statements]
expected_labels = [item[1] for item in paragraph_statements]

# Predict
predicted_labels = predict_sentiment(long_paragraphs, model, tokenizer, device)

# Evaluate and show
incorrect_count = 0

for text, expected, predicted in zip(long_paragraphs, expected_labels, predicted_labels):
    print(f"📝 Paragraph: {text}")
    print(f"✅ Expected: {expected}")
    print(f"🔎 Predicted: {predicted}")
    if expected.lower() != predicted.lower():
        print("❌ Incorrect Prediction\n")
        incorrect_count += 1
    else:
        print("✅ Correct Prediction\n")

# Summary
total = len(long_paragraphs)
print("📊 Paragraph Evaluation Summary:")
print(f"✅ Total Tested: {total}")
print(f"✔️ Correct: {total - incorrect_count}")
print(f"❌ Incorrect: {incorrect_count}")


📝 Paragraph: The company reported outstanding performance in the last quarter. Revenue grew by 30%, largely due to strong customer retention and increased demand for their flagship product. The leadership team announced bonuses and expanded hiring to meet growing needs. Analysts expressed confidence in the company's upward trajectory.
✅ Expected: Positive
🔎 Predicted: Positive
✅ Correct Prediction

📝 Paragraph: Following the successful launch of its new service, the company saw record-breaking user registrations within a week. Positive reviews flooded in, praising its user interface and performance. Investors responded with enthusiasm, pushing share prices higher. Executives confirmed further innovation in the pipeline.
✅ Expected: Positive
🔎 Predicted: Positive
✅ Correct Prediction

📝 Paragraph: The tech startup secured a major funding round from global investors. This will enable the company to scale its operations across multiple continents. The leadership emphasized sustainable gro